In [1]:
import pandas as pd
from pathlib import Path
import os
import glob
import re

In [2]:
HOME_DIR = Path.home() / 'code' / 'clir' / 'data' / 'wikidata' / 'base_data'
HOME_DIR

PosixPath('/Users/yanghe/code/clir/data/wikidata/base_data')

In [3]:
# --------------------- 过滤 query 的 QID ---------------------
QID_search_results_file = str(HOME_DIR / 'base_train_QID_search_results.csv')
QID_filtered_search_results_file = str(HOME_DIR / 'base_train_QID_filtered_search_results.csv')

def filter_qid(original_file: str, filtered_file: str):

    QID_df = pd.read_csv(original_file, encoding='utf-8')

    # columns_to_keep = ['query', 'search_term', 'id', 'label', 'description']
    columns_to_keep = ['query_id', 'query', 'qid']

    # 只取 columns_to_keep 列数据，然后去除 id 列 为NaN的行数据
    QID_filtered_df = QID_df[columns_to_keep].dropna(subset=['qid'])
    # 去除重复项
    QID_filtered_df.drop_duplicates(keep='first', inplace=True)

    # 保存文件
    QID_filtered_df.to_csv(filtered_file, index=False, encoding='utf-8')

# filter_qid(QID_search_results_file, QID_filtered_search_results_file)

In [4]:
# --------------------- 删除 实体、属性英文信息为空的行数据 ---------------------
def filter_item_info(original_file: str, filtered_file: str):

    item_info_df = pd.read_csv(original_file, encoding='utf-8')
    # 删除'label_en', 'description_en' 为空的行数据
    item_info_df = item_info_df.dropna(subset=['label_en', 'description_en'], how='any')

    item_info_df.to_csv(filtered_file, index=False, encoding='utf-8')

query_entity_info_file = str(HOME_DIR / 'base_train_query_entity_info.csv')
query_entity_filtered_info_file = str(HOME_DIR / 'base_train_query_entity_filtered_info.csv')

# filter_item_info(query_entity_info_file, query_entity_filtered_info_file)


In [5]:
df = pd.read_csv(query_entity_filtered_info_file)
df

,item_qid,label_zh,label_kk,label_en,description_zh,description_kk,description_en
0,Q381194,艾青,NaN,Ai Qing,中国诗人（1910－1996）,NaN,Chinese poet (1910-1996)
1,Q656448,“三个代表”重要思想,NaN,Theory of Three Represents,以江泽民为核心的中共领导层提出的政治理念,NaN,Political ideology formulated by Jiang Zemin
2,Q2726,7月19日,19 шілде,July 19,日期,NaN,date
3,Q588748,爆竹,NaN,firecracker,NaN,NaN,small explosive device primarily designed to m...
4,Q553678,葛洪,NaN,Ge Hong,NaN,NaN,4th century Chinese scholar and philosopher
...,...,...,...,...,...,...,...
6396,Q5358401,電子振盪,NaN,electronic oscillation,NaN,NaN,continuous recurrence of the same electrical p...
6397,Q3878252,波奧亞 (夏威夷州),NaN,Pauoa,NaN,NaN,"unincorporated community in Honolulu County, H..."
6398,Q5627116,居普菲山,NaN,Güpfi,瑞士山峰,NaN,mountain in Switzerland
6399,Q2048586,升变王车易位,NaN,Pam–Krabbé castling,特殊的王车易位方式，现在被国际棋联规则禁止,NaN,castling along e-file with a pawn that underpr...


In [7]:
df.iloc[0:100]

,item_qid,label_zh,label_kk,label_en,description_zh,description_kk,description_en
0,Q381194,艾青,NaN,Ai Qing,中国诗人（1910－1996）,NaN,Chinese poet (1910-1996)
1,Q656448,“三个代表”重要思想,NaN,Theory of Three Represents,以江泽民为核心的中共领导层提出的政治理念,NaN,Political ideology formulated by Jiang Zemin
2,Q2726,7月19日,19 шілде,July 19,日期,NaN,date
3,Q588748,爆竹,NaN,firecracker,NaN,NaN,small explosive device primarily designed to m...
4,Q553678,葛洪,NaN,Ge Hong,NaN,NaN,4th century Chinese scholar and philosopher
...,...,...,...,...,...,...,...
95,Q57394,努尔苏丹·纳扎尔巴耶夫,Нұрсұлтан Әбішұлы Назарбаев,Nursultan Nazarbayev,哈萨克斯坦首任总统 前哈萨克共产党中央第一书记,Қазақстанның мемлекеттік және саяси қайраткері...,President of Kazakhstan from 1990 to 2019
96,Q5253,1958年,1958 жыл,1958,年份,NaN,year
97,Q185203,金星凌日,NaN,transit of Venus,太陽和地球之間的金星遮蔽一小部分太陽對地輻射的天文現象,NaN,astronomical transit of Venus across the Sun
98,Q49101,第二次中东战争,NaN,Suez Crisis,1956年军事冲突,NaN,"1956 invasion of Egypt by Israel, the United K..."


In [9]:
df2 = df.iloc[100:300]
for index, row in df2.iterrows():
    print(index)
    break

100


In [3]:
a = [{'a': 's', 'b': 'f'}]
b = [{'a': 's', 'b': 'f'}, {'c': 'd', 'e': 'g'}]
c = [a, b]
c

[[{'a': 's', 'b': 'f'}], [{'a': 's', 'b': 'f'}, {'c': 'd', 'e': 'g'}]]

In [4]:
d = sum(c, [])
d

[{'a': 's', 'b': 'f'}, {'a': 's', 'b': 'f'}, {'c': 'd', 'e': 'g'}]